In [1]:
from subpred.dataset import create_dataset, get_tcdb_substrates, get_go_df, get_keywords_df
df_swissprot = create_dataset(
        # keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
        # keywords_component_filter=["Transmembrane"],
        # keywords_transport_filter=["Transport"],
        input_file="../data/raw/swissprot/uniprot_data_2022_04.tab.gz",
        multi_substrate="keep",
        verbose=True,
        # tax_ids_filter=[3702, 9606, 83333, 559292],
        # outliers=outliers,
        # sequence_clustering=70,
        evidence_code=1,
        invalid_amino_acids="remove_protein",
        tcdb_substrates_file="../data/raw/tcdb/tcdb_substrates.tsv"
        # force_update=True
    )


Found pickle, reading...


In [2]:
df_swissprot.tcdb_substrates[~df_swissprot.tcdb_substrates.isnull()]

Uniprot
A0A061ACU2                                    CHEBI:3473;cation
A0A1L8F5J9                                      CHEBI:24870;ion
A0FGR8                                         CHEBI:6486;lipid
A0FGR9                                   CHEBI:3308;calcium(2+)
A0PJK1                 CHEBI:9175;sodium(1+)|CHEBI:5418;glucose
                                    ...                        
Q9ZW74                                      CHEBI:14911;protein
P9WHY3        CHEBI:45422;propionamide|CHEBI:5448;glycerol|C...
Q01368                                      CHEBI:14911;protein
Q57134           CHEBI:75980;single-stranded DNA polynucleotide
Q58026                              CHEBI:2642;alpha-amino acid
Name: tcdb_substrates, Length: 3980, dtype: object

In [3]:
get_tcdb_substrates(df_swissprot)

,Uniprot,tcdb_id,chebi_id,chebi_term
0,A0A061ACU2,1.A.75.1.8,CHEBI:3473,cation
1,A0A1L8F5J9,1.A.10.1.12,CHEBI:24870,ion
2,A0FGR8,9.A.57.1.2,CHEBI:6486,lipid
3,A0FGR9,9.A.57.1.3,CHEBI:3308,calcium(2+)
4,A0PJK1,2.A.21.3.15,CHEBI:9175,sodium(1+)
...,...,...,...,...
6852,P9WHY3,1.B.94.1.1,CHEBI:5448,glycerol
6853,P9WHY3,1.B.94.1.1,CHEBI:5418,glucose
6854,Q01368,9.B.70.1.1,CHEBI:14911,protein
6855,Q57134,3.A.11.2.2,CHEBI:75980,single-stranded DNA polynucleotide


In [4]:
get_go_df(df_swissprot)

,Uniprot,go_id,go_term
0,A0A024SC78,GO:0005576,extracellular region
1,A0A024SC78,GO:0050525,cutinase activity
2,A0A024SH76,GO:0005576,extracellular region
3,A0A024SH76,GO:0016162,"cellulose 1,4-beta-cellobiosidase activity"
4,A0A024SH76,GO:0030248,cellulose binding
...,...,...,...
1028353,Q9ZUU2,GO:0009507,chloroplast
1028354,Q9ZUU2,GO:0016209,antioxidant activity
1028355,V5XVW4,GO:0019028,viral capsid
1028356,W5X2N3,NaN,NaN


In [5]:
get_keywords_df(df_swissprot)

,Uniprot,keyword
0,A0A024SC78,3D-structure
1,A0A024SC78,Disulfide bond
2,A0A024SC78,Hydrolase
3,A0A024SC78,Secreted
4,A0A024SC78,Serine esterase
...,...,...
831452,V5XVW4,Capsid protein
831453,V5XVW4,Direct protein sequencing
831454,V5XVW4,Virion
831455,W5X2N3,nan


In [16]:
print(df_swissprot.shape)
sum(~df_swissprot.tcdb_id.isnull())

(95197, 15)


5691

In [52]:
tcdb_substrates = "../data/raw/tcdb/tcdb_substrates.tsv"
import pandas as pd

df_substrates = pd.read_table(tcdb_substrates, header=None)
df_substrates.columns = ["tcdb", "substrates"]
substr = df_substrates.substrates.str.split("|").explode()
df_substrates = df_substrates.tcdb.to_frame().join(substr)
df_substrates = (
    df_substrates.tcdb.to_frame()
    .join(df_substrates.substrates.str.split(";", expand=True))
    .drop_duplicates()
)
df_substrates_long = df_substrates.rename(columns={0: "chebi", 1: "molecule"})

# df_substrates_wide = pd.concat(
#     [
#         df_substrates.groupby("tcdb").apply(lambda x: ";".join(x["chebi"])),
#         df_substrates.groupby("tcdb").apply(lambda x: ";".join(x["molecule"])),
#     ],
#     axis=1,
# )
# df_substrates_wide.columns = ["chebi", "molecule"]

# df_substrates_wide

In [7]:
from subpred.dataset import create_dataset
df_all = create_dataset(
        # keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
        # keywords_component_filter=["Transmembrane"],
        # keywords_transport_filter=["Transport"],
        input_file="../data/raw/swissprot/uniprot_data_all_evidence1_2022_04.tsv",
        multi_substrate="keep",
        verbose=True,
        # tax_ids_filter=[3702, 9606, 83333, 559292],
        # outliers=outliers,
        # sequence_clustering=70,
        evidence_code=1,
        invalid_amino_acids="remove_protein",
        # force_update=True
    )

Found pickle, reading...


In [8]:
df_all.shape

(242787, 15)

In [13]:
sum(~df_all.tcdb_id.isnull())

6187

## Keywords 

### Substrates

In [2]:
df_substrates = df[df.keywords_substrates != ""]
print(df_substrates.shape)
df_substrates.keywords_substrates.value_counts()

(14296, 15)


Protein transport                                                 3278
Electron transport                                                2139
Ion transport                                                     1057
Oxygen transport                                                   889
Hydrogen ion transport;Ion transport                               740
                                                                  ... 
Ion transport;Neurotransmitter transport;Sodium transport            1
Ion transport;Phosphate transport;Zinc transport                     1
Cobalt transport;Ion transport;Iron transport;Zinc transport         1
Cobalt transport;Copper transport;Ion transport;Zinc transport       1
Amino-acid transport;Ion transport                                   1
Name: keywords_substrates, Length: 87, dtype: int64

In [3]:
df_substrates_long = df_substrates.keywords_substrates.str.split(";").explode()

display(df_substrates_long.value_counts())
display(df_substrates_long.groupby())

Ion transport                         4487
Protein transport                     4114
Electron transport                    2189
Oxygen transport                       889
Hydrogen ion transport                 816
Lipid transport                        785
Sodium transport                       732
Potassium transport                    704
Sugar transport                        703
mRNA transport                         669
Translocation                          626
Amino-acid transport                   545
Calcium transport                      463
Chloride                               392
Iron transport                         293
Zinc transport                         191
Neurotransmitter transport             175
Phosphate transport                    143
Peptide transport                      119
Copper transport                        84
Ammonia transport                       79
Sodium/potassium transport              77
Anion exchange                          51
Sulfate tra

TypeError: You have to supply one of 'by' and 'level'

### Transmembrane transport

In [ ]:
df_keywords_long = df.keywords.str.split(";").explode().str.strip().reset_index(drop=False)

df_keywords_long = df_keywords_long.rename(columns = {"keywords": "keyword"})

df_keywords_long

,Uniprot,keyword
0,A0A024SC78,3D-structure
1,A0A024SC78,Disulfide bond
2,A0A024SC78,Hydrolase
3,A0A024SC78,Secreted
4,A0A024SC78,Serine esterase
...,...,...
1205125,V5XVW4,Capsid protein
1205126,V5XVW4,Direct protein sequencing
1205127,V5XVW4,Virion
1205128,W5X2N3,nan


In [ ]:
prot_arr_tm = df_keywords_long[df_keywords_long.keyword== "Transmembrane"].Uniprot.values
prot_arr_tp = df_keywords_long[df_keywords_long.keyword== "Transport"].Uniprot.values
import numpy as np

prot_arr_tm_tp = np.intersect1d(prot_arr_tm, prot_arr_tp)

prot_arr_tm_tp.size

9624

## Gene ontology (Uniprot column annotation)

Relevant terms:

- transmembrane transporter activity: https://www.ebi.ac.uk/QuickGO/term/GO:0022857
- membrane: https://www.ebi.ac.uk/QuickGO/term/GO:0016020
- plasma membrane: https://www.ebi.ac.uk/QuickGO/term/GO:0005886
- active transmembrane transporter activity
    - primary active transmembrane transporter activity
    - secondary active transmembrane transporter activity
    - etc.

TODO 

- does dataset contain all parent nodes?
- stats on active transport etc.
- pay attention to type of relationship (is_a), and evidence codes
- Which go terms are included in uniprot column? are they filtered?
- get go terms from GO directly, or from QuickGO (limited to 2mio annotations)

In [ ]:
# Set of relevant GO terms:

transmembrane_transporter_activity = {
    "GO:0022857",
    "GO:0005386",
    "GO:0015563",
    "GO:0015646",
    "GO:0022891",
    "GO:0022892",
}


In [ ]:
import pandas as pd
import re

df_go = df[~df.go_terms.isnull()]
df_go = df_go.go_terms.str.split(";").explode().str.strip().reset_index(drop=False)
go_id_pattern = re.compile("\[(GO\:[0-9]{7})\]")
df_go["go_id"] = df_go.go_terms.str.extract(go_id_pattern)
df_go["go_term"] = df_go.go_terms.str.replace(go_id_pattern, "").str.strip()
df_go = df_go.drop("go_terms", axis=1)
df_go = df_go.drop_duplicates()

In [ ]:
df_go[df_go.go_term == "transmembrane transporter activity"]

,Uniprot,go_id,go_term
627,A0A0B7P9G0,GO:0022857,transmembrane transporter activity
1746,A0A131MCZ8,GO:0022857,transmembrane transporter activity
4569,A1A5C7,GO:0022857,transmembrane transporter activity
5484,A1Z8N1,GO:0022857,transmembrane transporter activity
6841,A2AJN7,GO:0022857,transmembrane transporter activity
...,...,...,...
1394142,Q9XI74,GO:0022857,transmembrane transporter activity
1394689,Q9ZUS1,GO:0022857,transmembrane transporter activity
1394731,Q9ZVH5,GO:0022857,transmembrane transporter activity
1394779,S0ECK8,GO:0022857,transmembrane transporter activity


In [ ]:
obo = get_namespace("http://purl.obolibrary.org/obo/")